In [32]:
import pandas as pd
import numpy as np
import requests
import geopandas as gpd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import sqlite3 as sq
pd.options.mode.chained_assignment = None

In [33]:
def mantissa_round(x):
    y = np.floor(x)
    indices = np.argsort(x-y)
    hilowman = indices[::-1]
    diffxy = np.sum(x)-np.sum(y)
    z=0
    while z < int(diffxy):
        y[hilowman[z]] +=1
        z +=1
    return np.array(y, dtype='int')



In [34]:
dist = pd.read_csv('../data/LEHDDistributionnewgroups.csv')
dist['County'] = dist['County'].str.split(pat = ',', expand = True)[0]
dist.head(2)

,GEO_ID,County,Total Jobs %,Industrial %,Office %,Service %,Other %,Education %,Food Services %,Government %,Medical %,Retail %
0,470210701021001,Cheatham County,0.009725,0.0,0.000000,0.0,0.004188,0.0,0.0,0.020305,0.115318,0.0
1,470210701021004,Cheatham County,0.002093,0.0,0.007009,0.0,0.010471,0.0,0.0,0.000000,0.001721,0.0


In [35]:
dist['GEO_ID'] = dist['GEO_ID'].astype(str)
c_inverted = dict(zip(dist['GEO_ID'], dist['County']))

In [36]:
bl = pd.read_csv('../data/aggregatedinds_newestgroupsfordataaxel_rounded.csv')
bl['County'] = bl['County'].str.split(pat = ',', expand = True)[0]
bl = bl.set_index('County')
bl = bl.add_suffix(' County Total').reset_index(drop = False)

In [37]:
bl.head(2)

,County,Education County Total,Food Services County Total,Government County Total,Industrial County Total,Medical County Total,Office County Total,Other County Total,Retail County Total,Service County Total,Total employment (number of jobs) County Total
0,Cheatham County,519,945,1209,4306,833,3771,2528,1516,1741,17368
1,Davidson County,38447,59595,26262,100561,97834,253596,43554,51863,68622,740334


In [38]:
data = bl.merge(dist, on = 'County', how = 'outer')

In [39]:
industries = ['Industrial', 'Office', 'Service', 
             'Other', 'Education', 'Food Services', 'Government', 'Medical', 'Retail']
for ind in industries:
    data['{}'.format(ind)] = data['{} County Total'.format(ind)] * data['{} %'.format(ind)]

In [40]:
data.head(2)

,County,Education County Total,Food Services County Total,Government County Total,Industrial County Total,Medical County Total,Office County Total,Other County Total,Retail County Total,Service County Total,Total employment (number of jobs) County Total,GEO_ID,Total Jobs %,Industrial %,Office %,Service %,Other %,Education %,Food Services %,Government %,Medical %,Retail %,Industrial,Office,Service,Other,Education,Food Services,Government,Medical,Retail
0,Cheatham County,519,945,1209,4306,833,3771,2528,1516,1741,17368,470210701021001,0.009725,0.0,0.000000,0.0,0.004188,0.0,0.0,0.020305,0.115318,0.0,0.0,0.000000,0.0,10.588482,0.0,0.0,24.548223,96.060241,0.0
1,Cheatham County,519,945,1209,4306,833,3771,2528,1516,1741,17368,470210701021004,0.002093,0.0,0.007009,0.0,0.010471,0.0,0.0,0.000000,0.001721,0.0,0.0,26.432243,0.0,26.471204,0.0,0.0,0.000000,1.433735,0.0


In [41]:
data = data[['County', 'GEO_ID', 'Industrial', 'Office', 'Service', 
             'Other', 'Education', 'Food Services', 'Government', 'Medical', 'Retail']]

In [42]:
data.head()

,County,GEO_ID,Industrial,Office,Service,Other,Education,Food Services,Government,Medical,Retail
0,Cheatham County,470210701021001,0.00000,0.000000,0.0,10.588482,0.0,0.0,24.548223,96.060241,0.000000
1,Cheatham County,470210701021004,0.00000,26.432243,0.0,26.471204,0.0,0.0,0.000000,1.433735,0.000000
2,Cheatham County,470210701021009,1.38948,0.000000,0.0,2.647120,0.0,0.0,0.000000,0.000000,1.667767
3,Cheatham County,470210701021012,0.00000,8.810748,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000
4,Cheatham County,470210701021013,0.00000,8.810748,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000


In [43]:
cheatham = data.loc[data['County'] == 'Cheatham County']
davidson = data.loc[data['County'] == 'Davidson County']
dickson = data.loc[data['County'] == 'Dickson County']
houston = data.loc[data['County'] == 'Houston County']
humphreys = data.loc[data['County'] == 'Humphreys County']
maury = data.loc[data['County'] == 'Maury County']
montgomery = data.loc[data['County'] == 'Montgomery County']
robertson = data.loc[data['County'] == 'Robertson County']
rutherford = data.loc[data['County'] == 'Rutherford County']
stewart = data.loc[data['County'] == 'Stewart County']
sumner = data.loc[data['County'] == 'Sumner County']
trousdale = data.loc[data['County'] == 'Trousdale County']
williamson = data.loc[data['County'] == 'Williamson County']
wilson = data.loc[data['County'] == 'Wilson County']

In [44]:
# dfs = [cheatham, davidson, dickson, houston, humphreys, maury, montgomery, robertson, rutherford, stewart, sumner, trousdale, williamson, wilson]
# inds = ['Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 'Manufacturing', 
#         'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 'Real estate and rental and leasing', 
#         'Professional, scientific, and technical services', 'Management of companies and enterprises', 
#         'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
#         'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
#         'Public Administration']
# for df in dfs:
#     data = df
#     data = data[['GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
#               'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
#               'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
#               'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
#               'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
#               'Public Administration']]
#     for ind in inds:
#         data['{} Mantissa'.format(ind)] = mantissa_round(data['{}'.format(ind)])
#         df = data

In [45]:
# cheatham = cheatham[['GEO_ID', 'Agriculture, forestry, fishing and hunting', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 'Construction', 
#               'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation and warehousing', 'Information', 'Finance and insurance', 
#               'Real estate and rental and leasing', 'Professional, scientific, and technical services', 'Management of companies and enterprises', 
#               'Administrative and support and waste management and remediation services', 'Educational services', 'Health care and social assistance', 
#               'Arts, entertainment, and recreation', 'Accommodation and food services', 'Other services (except government and government enterprises)', 
#               'Public Administration']]
# for ind in inds:
#     cheatham['{} Mantissa'.format(ind)] = mantissa_round(data['{}'.format(ind)])

In [46]:
cheatham = cheatham[['GEO_ID', 'Industrial', 'Office', 'Service', 
             'Other', 'Education', 'Food Services', 'Government', 'Medical', 'Retail']]
cheatham.set_index('GEO_ID', inplace = True)
inds = list(cheatham.columns)
cheatham.reset_index(drop = False, inplace = True)
cheatham.fillna(0, inplace = True)
for ind in inds:
    cheatham['{} Mantissa'.format(ind)] = mantissa_round(cheatham['{}'.format(ind)])
cheatham['County'] = 'Cheatham County'

In [47]:
diff = cheatham.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Industrial Diff', 'Office Diff', 'Service Diff', 
             'Other Diff', 'Education Diff', 'Food Services Diff', 'Government Diff', 'Medical Diff', 'Retail Diff']]
diff1 = diff.reset_index(drop = False)
diff1.head()

,County,Industrial Diff,Office Diff,Service Diff,Other Diff,Education Diff,Food Services Diff,Government Diff,Medical Diff,Retail Diff
0,Cheatham County,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [48]:
davidson = davidson[['GEO_ID', 'Industrial', 'Office', 'Service', 
             'Other', 'Education', 'Food Services', 'Government', 'Medical', 'Retail']]
davidson.set_index('GEO_ID', inplace = True)
inds = list(davidson.columns)
davidson.reset_index(drop = False, inplace = True)
davidson.fillna(0, inplace = True)
for ind in inds:
    davidson['{} Mantissa'.format(ind)] = mantissa_round(davidson['{}'.format(ind)])
davidson['County'] = 'Davidson County'

In [49]:
diff = davidson.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Industrial Diff', 'Office Diff', 'Service Diff', 
             'Other Diff', 'Education Diff', 'Food Services Diff', 'Government Diff', 'Medical Diff', 'Retail Diff']]
diff2 = diff.reset_index(drop = False)
diff2.head()

,County,Industrial Diff,Office Diff,Service Diff,Other Diff,Education Diff,Food Services Diff,Government Diff,Medical Diff,Retail Diff
0,Davidson County,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [50]:
dickson = dickson[['GEO_ID', 'Industrial', 'Office', 'Service', 
             'Other', 'Education', 'Food Services', 'Government', 'Medical', 'Retail']]
dickson.set_index('GEO_ID', inplace = True)
inds = list(dickson.columns)
dickson.reset_index(drop = False, inplace = True)
dickson.fillna(0, inplace = True)
for ind in inds:
    dickson['{} Mantissa'.format(ind)] = mantissa_round(dickson['{}'.format(ind)])
dickson['County'] = 'Dickson County'

In [51]:
diff = dickson.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Industrial Diff', 'Office Diff', 'Service Diff', 
             'Other Diff', 'Education Diff', 'Food Services Diff', 'Government Diff', 'Medical Diff', 'Retail Diff']]
diff3 = diff.reset_index(drop = False)
diff3.head()

,County,Industrial Diff,Office Diff,Service Diff,Other Diff,Education Diff,Food Services Diff,Government Diff,Medical Diff,Retail Diff
0,Dickson County,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [52]:
houston = houston[['GEO_ID', 'Industrial', 'Office', 'Service', 
             'Other', 'Education', 'Food Services', 'Government', 'Medical', 'Retail']]
houston.set_index('GEO_ID', inplace = True)
inds = list(houston.columns)
houston.reset_index(drop = False, inplace = True)
houston.fillna(0, inplace = True)
for ind in inds:
    houston['{} Mantissa'.format(ind)] = mantissa_round(houston['{}'.format(ind)])
houston['County'] = 'Houston County'

In [53]:
diff = houston.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = (diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)])
diff = diff[['Industrial Diff', 'Office Diff', 'Service Diff', 
             'Other Diff', 'Education Diff', 'Food Services Diff', 'Government Diff', 'Medical Diff', 'Retail Diff']]
diff4 = diff.reset_index(drop = False)
diff4.head()

,County,Industrial Diff,Office Diff,Service Diff,Other Diff,Education Diff,Food Services Diff,Government Diff,Medical Diff,Retail Diff
0,Houston County,1.0,1.0,1.0,1.0,-3.552714e-15,1.0,0.0,1.0,1.0


In [54]:
humphreys = humphreys[['GEO_ID', 'Industrial', 'Office', 'Service', 
             'Other', 'Education', 'Food Services', 'Government', 'Medical', 'Retail']]
humphreys.set_index('GEO_ID', inplace = True)
inds = list(humphreys.columns)
humphreys.reset_index(drop = False, inplace = True)
humphreys.fillna(0, inplace = True)
for ind in inds:
    humphreys['{} Mantissa'.format(ind)] = mantissa_round(humphreys['{}'.format(ind)])
humphreys['County'] = 'Humphreys County'

In [55]:
diff = humphreys.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Industrial Diff', 'Office Diff', 'Service Diff', 
             'Other Diff', 'Education Diff', 'Food Services Diff', 'Government Diff', 'Medical Diff', 'Retail Diff']]
diff5 = diff.reset_index(drop = False)
diff5.head()

,County,Industrial Diff,Office Diff,Service Diff,Other Diff,Education Diff,Food Services Diff,Government Diff,Medical Diff,Retail Diff
0,Humphreys County,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [56]:
maury = maury[['GEO_ID', 'Industrial', 'Office', 'Service', 
             'Other', 'Education', 'Food Services', 'Government', 'Medical', 'Retail']]
maury.set_index('GEO_ID', inplace = True)
inds = list(maury.columns)
maury.reset_index(drop = False, inplace = True)
maury.fillna(0, inplace = True)
for ind in inds:
    maury['{} Mantissa'.format(ind)] = mantissa_round(maury['{}'.format(ind)])
maury['County'] = 'Maury County'

In [57]:
diff = maury.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Industrial Diff', 'Office Diff', 'Service Diff', 
             'Other Diff', 'Education Diff', 'Food Services Diff', 'Government Diff', 'Medical Diff', 'Retail Diff']]
diff6 = diff.reset_index(drop = False)
diff6.head()

,County,Industrial Diff,Office Diff,Service Diff,Other Diff,Education Diff,Food Services Diff,Government Diff,Medical Diff,Retail Diff
0,Maury County,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [58]:
montgomery = montgomery[['GEO_ID', 'Industrial', 'Office', 'Service', 
             'Other', 'Education', 'Food Services', 'Government', 'Medical', 'Retail']]
montgomery.set_index('GEO_ID', inplace = True)
inds = list(montgomery.columns)
montgomery.reset_index(drop = False, inplace = True)
montgomery.fillna(0, inplace = True)
for ind in inds:
    montgomery['{} Mantissa'.format(ind)] = mantissa_round(montgomery['{}'.format(ind)])
montgomery['County'] = 'Montgomery County'

In [59]:
diff = montgomery.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Industrial Diff', 'Office Diff', 'Service Diff', 
             'Other Diff', 'Education Diff', 'Food Services Diff', 'Government Diff', 'Medical Diff', 'Retail Diff']]
diff7 = diff.reset_index(drop = False)
diff7.head()

,County,Industrial Diff,Office Diff,Service Diff,Other Diff,Education Diff,Food Services Diff,Government Diff,Medical Diff,Retail Diff
0,Montgomery County,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [60]:
robertson = robertson[['GEO_ID', 'Industrial', 'Office', 'Service', 
             'Other', 'Education', 'Food Services', 'Government', 'Medical', 'Retail']]
robertson.set_index('GEO_ID', inplace = True)
inds = list(robertson.columns)
robertson.reset_index(drop = False, inplace = True)
robertson.fillna(0, inplace = True)
for ind in inds:
    robertson['{} Mantissa'.format(ind)] = mantissa_round(robertson['{}'.format(ind)])
robertson['County'] = 'Robertson County'

In [61]:
diff = robertson.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Industrial Diff', 'Office Diff', 'Service Diff', 
             'Other Diff', 'Education Diff', 'Food Services Diff', 'Government Diff', 'Medical Diff', 'Retail Diff']]
diff8 = diff.reset_index(drop = False)
diff8.head()

,County,Industrial Diff,Office Diff,Service Diff,Other Diff,Education Diff,Food Services Diff,Government Diff,Medical Diff,Retail Diff
0,Robertson County,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0


In [62]:
rutherford = rutherford[['GEO_ID', 'Industrial', 'Office', 'Service', 
             'Other', 'Education', 'Food Services', 'Government', 'Medical', 'Retail']]
rutherford.set_index('GEO_ID', inplace = True)
inds = list(rutherford.columns)
rutherford.reset_index(drop = False, inplace = True)
rutherford.fillna(0, inplace = True)
for ind in inds:
    rutherford['{} Mantissa'.format(ind)] = mantissa_round(rutherford['{}'.format(ind)])
rutherford['County'] = 'Rutherford County'

In [63]:
diff = rutherford.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Industrial Diff', 'Office Diff', 'Service Diff', 
             'Other Diff', 'Education Diff', 'Food Services Diff', 'Government Diff', 'Medical Diff', 'Retail Diff']]
diff9 = diff.reset_index(drop = False)
diff9.head()

,County,Industrial Diff,Office Diff,Service Diff,Other Diff,Education Diff,Food Services Diff,Government Diff,Medical Diff,Retail Diff
0,Rutherford County,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [64]:
stewart = stewart[['GEO_ID', 'Industrial', 'Office', 'Service', 
             'Other', 'Education', 'Food Services', 'Government', 'Medical', 'Retail']]
stewart.set_index('GEO_ID', inplace = True)
inds = list(stewart.columns)
stewart.reset_index(drop = False, inplace = True)
stewart.fillna(0, inplace = True)
for ind in inds:
    stewart['{} Mantissa'.format(ind)] = mantissa_round(stewart['{}'.format(ind)])
stewart['County'] = 'Stewart County'

In [65]:
diff = stewart.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Industrial Diff', 'Office Diff', 'Service Diff', 
             'Other Diff', 'Education Diff', 'Food Services Diff', 'Government Diff', 'Medical Diff', 'Retail Diff']]
diff10 = diff.reset_index(drop = False)
diff10.head()

,County,Industrial Diff,Office Diff,Service Diff,Other Diff,Education Diff,Food Services Diff,Government Diff,Medical Diff,Retail Diff
0,Stewart County,1.0,1.0,1.0,1.0,-7.105427e-15,1.0,1.0,1.0,1.0


In [66]:
sumner = sumner[['GEO_ID', 'Industrial', 'Office', 'Service', 
             'Other', 'Education', 'Food Services', 'Government', 'Medical', 'Retail']]
sumner.set_index('GEO_ID', inplace = True)
inds = list(sumner.columns)
sumner.reset_index(drop = False, inplace = True)
sumner.fillna(0, inplace = True)
for ind in inds:
    sumner['{} Mantissa'.format(ind)] = mantissa_round(sumner['{}'.format(ind)])
sumner['County'] = 'Sumner County'

In [67]:
diff = sumner.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Industrial Diff', 'Office Diff', 'Service Diff', 
             'Other Diff', 'Education Diff', 'Food Services Diff', 'Government Diff', 'Medical Diff', 'Retail Diff']]
diff11 = diff.reset_index(drop = False)
diff11.head()

,County,Industrial Diff,Office Diff,Service Diff,Other Diff,Education Diff,Food Services Diff,Government Diff,Medical Diff,Retail Diff
0,Sumner County,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [68]:
trousdale = trousdale[['GEO_ID', 'Industrial', 'Office', 'Service', 
             'Other', 'Education', 'Food Services', 'Government', 'Medical', 'Retail']]
trousdale.set_index('GEO_ID', inplace = True)
inds = list(trousdale.columns)
trousdale.reset_index(drop = False, inplace = True)
trousdale.fillna(0, inplace = True)
for ind in inds:
    trousdale['{} Mantissa'.format(ind)] = mantissa_round(trousdale['{}'.format(ind)])
trousdale['County'] = 'Trousdale County'

In [69]:
diff = trousdale.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Industrial Diff', 'Office Diff', 'Service Diff', 
             'Other Diff', 'Education Diff', 'Food Services Diff', 'Government Diff', 'Medical Diff', 'Retail Diff']]
diff12 = diff.reset_index(drop = False)
diff12.head()

,County,Industrial Diff,Office Diff,Service Diff,Other Diff,Education Diff,Food Services Diff,Government Diff,Medical Diff,Retail Diff
0,Trousdale County,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0


In [70]:
williamson = williamson[['GEO_ID', 'Industrial', 'Office', 'Service', 
             'Other', 'Education', 'Food Services', 'Government', 'Medical', 'Retail']]
williamson.set_index('GEO_ID', inplace = True)
inds = list(williamson.columns)
williamson.reset_index(drop = False, inplace = True)
williamson.fillna(0, inplace = True)
for ind in inds:
    williamson['{} Mantissa'.format(ind)] = mantissa_round(williamson['{}'.format(ind)])
williamson['County'] = 'Williamson County'

In [71]:
diff = williamson.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Industrial Diff', 'Office Diff', 'Service Diff', 
             'Other Diff', 'Education Diff', 'Food Services Diff', 'Government Diff', 'Medical Diff', 'Retail Diff']]
diff13 = diff.reset_index(drop = False)
diff13.head()

,County,Industrial Diff,Office Diff,Service Diff,Other Diff,Education Diff,Food Services Diff,Government Diff,Medical Diff,Retail Diff
0,Williamson County,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [72]:
wilson = wilson[['GEO_ID', 'Industrial', 'Office', 'Service', 
             'Other', 'Education', 'Food Services', 'Government', 'Medical', 'Retail']]
wilson.set_index('GEO_ID', inplace = True)
inds = list(wilson.columns)
wilson.reset_index(drop = False, inplace = True)
wilson.fillna(0, inplace = True)
for ind in inds:
    wilson['{} Mantissa'.format(ind)] = mantissa_round(wilson['{}'.format(ind)])
wilson['County'] = 'Wilson County'

In [73]:
diff = wilson.drop(columns = 'GEO_ID').groupby('County').sum()
for ind in inds:
    diff['{} Diff'.format(ind)] = diff['{}'.format(ind)] - diff['{} Mantissa'.format(ind)]
diff = diff[['Industrial Diff', 'Office Diff', 'Service Diff', 
             'Other Diff', 'Education Diff', 'Food Services Diff', 'Government Diff', 'Medical Diff', 'Retail Diff']]
diff14 = diff.reset_index(drop = False)
diff14.head()

,County,Industrial Diff,Office Diff,Service Diff,Other Diff,Education Diff,Food Services Diff,Government Diff,Medical Diff,Retail Diff
0,Wilson County,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [74]:
diffs = [diff1, diff2, diff3, diff4, diff5, diff6, diff7, diff8, diff9, diff10, diff11, diff12, diff13, diff14]
diff = pd.concat(diffs)

In [75]:
diff.to_csv('../data/diffs_newgroups.csv')

In [76]:
dfs = [cheatham, davidson, dickson, houston, humphreys, maury, montgomery, robertson, rutherford, stewart, sumner, trousdale, williamson, wilson]
data = pd.concat(dfs)

In [77]:
data['GEO_ID'] = data['GEO_ID'].astype(str)
#data['County'] = data['GEO_ID'].map(c_inverted)

In [78]:
data = data[['County', 'GEO_ID', 'Industrial Mantissa', 'Office Mantissa', 'Service Mantissa', 
             'Other Mantissa', 'Education Mantissa', 'Food Services Mantissa', 'Government Mantissa', 'Medical Mantissa', 'Retail Mantissa']]

In [79]:
data['GEO_ID'] = 'G'+ data['GEO_ID']

In [80]:
data.head()

,County,GEO_ID,Industrial Mantissa,Office Mantissa,Service Mantissa,Other Mantissa,Education Mantissa,Food Services Mantissa,Government Mantissa,Medical Mantissa,Retail Mantissa
0,Cheatham County,G470210701021001,1,1,1,10,1,1,24,96,1
1,Cheatham County,G470210701021004,0,26,0,26,0,0,0,1,0
2,Cheatham County,G470210701021009,1,0,0,2,0,0,0,0,1
3,Cheatham County,G470210701021012,0,8,0,0,0,0,0,0,0
4,Cheatham County,G470210701021013,0,8,0,0,0,0,0,0,0


In [81]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12040 entries, 0 to 885
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   County                  12040 non-null  object
 1   GEO_ID                  12040 non-null  object
 2   Industrial Mantissa     12040 non-null  int32 
 3   Office Mantissa         12040 non-null  int32 
 4   Service Mantissa        12040 non-null  int32 
 5   Other Mantissa          12040 non-null  int32 
 6   Education Mantissa      12040 non-null  int32 
 7   Food Services Mantissa  12040 non-null  int32 
 8   Government Mantissa     12040 non-null  int32 
 9   Medical Mantissa        12040 non-null  int32 
 10  Retail Mantissa         12040 non-null  int32 
dtypes: int32(9), object(2)
memory usage: 705.5+ KB


In [84]:
data.to_csv('../data/mantissa_newgroups_blockgroups2022.csv', index = False)